# Naamswijzgingen importeren en doorvoeren

Dit notebook wordt gebruikt om wijzigingen in naamgeving door te voeren voor
1. objecttypes
2. attributen

## Werking Import

De importmodule maakt gebruik van het feit dat het GGM is vastgelegd in Enterprise Architect van Sparx en dat deze de repository van het GGM in [Sqllite](https://www.sqlite.org/index.html) wordt bijgehouden. Deze importmodule leest de te wijzigen namen en past ze aan in de repository.

## Bedrijfsregels Import

Bij het importeren van vanuit Excels met wijzigingen vindt plaats conform 

1. Matching vindt plaats op basis van ggm-guid;
2. de naam wordt overschreven met de nieuw naam


In [23]:
### Importeer bibliotheken en utils

import os
import pandas as pd
import numpy as np
import json
from IPython.display import JSON as JSONDisplay
import requests
import database
import util
from re import sub
import ast
import re
from treelib import Node, Tree
from datetime import datetime
import math 

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import create_engine, select, MetaData, Table, and_, update, Sequence
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import declarative_base, Session
import uuid



def updateSequence(session, sequence):
    t_seq = session.query(Sequence).filter_by(name=sequence).first()
    newval = int(t_seq.seq) + 1
    
    stmt = update(Sequence).where(Sequence.c.name == sequence).values(seq=newval)
    session.execute(stmt)
    return newval


def generateEAUUID():
    return '{' +  str(uuid.uuid4()).upper() + '}'

def isEmpty(val):
    if not val:
        return True
    elif pd.isna(val):
        return True
    else:
        return False

def setAtrrIfNotEmpty(obj, prop, val):
    if not isEmpty(val):
        setattr(obj, prop, val)


## Configuratie

Met de waarden hieronder configureer je de export. Met root_guid geef je de root in de repository weer waar naartoe (en alles eronder) geïmporteerd wordt. Op deze manier kun je ook naar deelmodellen importeren, en alles negeren dat niet bij dat deelmodel hoort. Als voorbeeld zijn naast de root van het hele GGM ook Onderwijs en monumenten getoond.  

In [24]:
### Configuratie
db_uri = "sqlite:///root/Gemeentelijk Gegevensmodel EA16.qea"

now = datetime.now()
dt_string = now.strftime("%d%m%Y-%H:%M:%S")
import_objects = f"./input/ggm_objecttypen_gewijzigd.xlsx"
import_attributes = f"./input/ggm_attributen_gewijzigd.xlsx"

prop_original = 'originele-naam'

## Inlezen Wijzigingen Objecttypes

Hieronder worden alle wijzgingen ingelezen.

In [25]:
# Alle packages inlezen en omzetten in tree structure

df_objects_change = pd.read_excel(import_objects)
df_objects_change.head()

,id,alias,archimate_type,author,bron,created,datum_tijd_export,descr,domein_dcat,domein_gemma,...,name,package_id,phase,status,stereotype,synoniemen,toelichting,uri,version,visibility
0,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,Data object,abrienen,NaN,2019-12-18 13:52:42,28062023-11:06:06,Persoon waarmee communicatie plaatsvindt,NaN,NaN,...,Contact,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
1,EAID_5805FA72_C0CE_43c1_A53F_B81166AEFDD4,NaN,Business object,abrienen,NaN,2019-12-18 14:20:03,28062023-11:06:06,Gebouw waar je tegen betaling kunt logeren.,NaN,NaN,...,Hotel,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
2,EAID_669000E9_25D6_4346_B718_516CDB8B88B7,NaN,Business object,abrienen,NaN,2019-12-18 14:09:16,28062023-11:06:06,Verblijf in een hotel,NaN,NaN,...,Hotelbezoek,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
3,EAID_8F05E7BA_E7ED_45d1_9626_C8D7FBDB2F1B,NaN,Business object,abrienen,NaN,2019-12-18 14:19:47,28062023-11:06:06,Locatie waar iets wordt verkocht,NaN,NaN,...,Verkooppunt,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
4,EAID_EB35F5B8_9289_49cc_8DF4_8BD20EC662A9,NaN,Business object,abrienen,https://www.cbs.nl/nl-nl/onze-diensten/methode...,2019-12-18 14:08:27,28062023-11:06:06,"De vraag naar arbeid, te berekenen door de tot...",NaN,NaN,...,Werkgelegenheid,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN


In [27]:


# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)
metadata = MetaData(bind=None)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
Base = automap_base()
Base.prepare(engine, reflect=True)
Object = Base.classes.t_object
Objectproperty = Base.classes.t_objectproperties
Sequence = Table(
    'sqlite_sequence', 
    metadata, 
    autoload=True, 
    autoload_with=engine
)


# Do transformation
session = Session(engine)
var = None
try:
    for index, row in df_objects_change.iterrows():
        ea_guid = '{'+row['id'].replace("EAID_", "").replace("_", "-")+'}'
        t_obj = session.query(Object).filter_by(ea_guid=ea_guid).first()
        if t_obj and t_obj.Object_Type in ['Class','Enumeration'] and t_obj.Name != row['name']:

            # Oude naam wordt als property bewaard
            t_objprop = session.query(Objectproperty).filter_by(Object_ID=t_obj.Object_ID, Property=prop_original).first()
            if not t_objprop:
                #print(f'Property: {prop}')
                seq = updateSequence(session, "t_objectproperties")
                session.add(Objectproperty(PropertyID=seq, Object_ID=t_obj.Object_ID, Property=prop_original, Value=str(t_obj.Name) if not isEmpty(t_obj.Name) else '', ea_guid=generateEAUUID()))
            else:
                t_objprop.Value = t_obj.Name

            
            # Set Object value
            #print(f"Setting Oud: {t_obj.Name} to Nieuw: {row['name']}") 
            t_obj.Name = row['name']
        
        #print(t_obj.Name)
except Exception as e:
    print(f'Foutmelding {ex}')
    raise
        
finally:
    session.commit()
    engine.dispose()


## Inlezen Wijzigingen Attributen

Hieronder worden alle wijzgingen ingelezen.



In [28]:
# Alle packages inlezen en omzetten in tree structure

df_attrs_change = pd.read_excel(import_attributes)
df_attrs_change.head()

/opt/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,id,clazz_id,descr,enumeration_id,name_oud,name,primitive,type_class_id
0,EAID_B4F60A84_AFBB_4554_9887_DF1020E99DB8,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,contactsoort,contactsoort,NaN,NaN
1,EAID_DC8B5E9A_144A_4c78_ACAB_908E8DC2C757,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,datum,datum,NaN,NaN
2,EAID_B8A47DC2_CE92_4e1b_949B_BC2212AC341C,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,tekst,tekst,NaN,NaN
3,EAID_dst36212C_BD57_42b6_B51D_E49EA2853F66,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,NaN,NaN,NaN,NaN
4,EAID_dst4AC804_79E6_40a6_992F_C50B4F045C90,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)
metadata = MetaData(bind=None)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine, reflect=True) #Base.prepare(engine, reflect=True)
Attribute = Base.classes.t_attribute
Object = Base.classes.t_object
XRef = Base.classes.t_xref
AttributeTag = Base.classes.t_attributetag

Sequence = Table(
    'sqlite_sequence', 
    metadata, 
    autoload=True, 
    autoload_with=engine
)


# Do transformation
session = Session(engine)

for index, row in df_attrs_change.iterrows():
    ea_guid = '{'+row['id'].replace("EAID_", "").replace("_", "-")+'}'
    t_attr = session.query(Attribute).filter_by(ea_guid=ea_guid).first()
    if t_attr and t_attr.Name != row['name']:
        
        t_attrprop = session.query(AttributeTag).filter_by(ElementID=t_attr.ID, Property=prop_original).first()
        if not t_attrprop:
            seq = updateSequence(session, "t_attributetag")
            session.add(AttributeTag(PropertyID=seq, ElementID=t_attr.ID, Property=prop_original, VALUE=str(t_attr.Name), ea_guid=generateEAUUID()))
        else:
            t_attrprop.Value = t_attr.Name

        # Set Object value
        print(f"Setting Oud: {t_attr.Name} to Nieuw: {row['name']}") 
        t_attr.Name = row['name']

    #print(t_obj.Name)
    
session.commit()
engine.dispose()
                


Setting Oud: aantal kamers to Nieuw: aantalKamers
Setting Oud: einddatum to Nieuw: datumEind
Setting Oud: startdatum to Nieuw: datumStart
Setting Oud: aantalFulletimeVrouwen to Nieuw: aantalFulltimeVrouwen
Setting Oud: aantalkassa to Nieuw: aantalKassa
Setting Oud: BronWVO to Nieuw: bronWVO
Setting Oud: Wvoklasse to Nieuw: wvoKlasse
Setting Oud: eindeaanwezigheid to Nieuw: eindeAanwezigheid
Setting Oud: aanstellingsdatum to Nieuw: datumAanstelling
Setting Oud: uittredingsdatum to Nieuw: datumUittreding
Setting Oud: aanstellingsdatum to Nieuw: datumAanstelling
Setting Oud: uittredingsdatum to Nieuw: datumUittreding
Setting Oud: expiratiedatum to Nieuw: datumExpiratie
Setting Oud: publicatiedatum to Nieuw: datumPublicatie
Setting Oud: registratiedatum to Nieuw: datumRegistratie
Setting Oud: typeraadsstuk to Nieuw: typeRaadsstuk
Setting Oud: aanvraagdatum to Nieuw: datumAanvraag
Setting Oud: buitenlandse schoollocatie to Nieuw: buitenlandseSchoollocatie
Setting Oud: datum retour to Nieuw: